# Graph algorithms with Kùzu and NetworkX

In this notebook, we will cover how to run a graph algorithm to perform a meaningful analysis of graph data.
We will continue with the existing graph of a financial network. First, connect to the existing database.

> **Note**: Kùzu will soon support a native graph algorithms package, enabling you to run graph algorithms
> directly on the Kùzu graph via its Cypher interface. However, few libraries have the extensive set of
> graph algorithms that NetworkX does, so NetworkX will always be a great tool to run a whole suite of graph
> algorithms on top of your Kùzu graphs.

This notebook will show an example of how you can run the
[Betweenness Centrality](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html) algorithm from NetworkX on the financial network graph.

In [1]:
# !pip install kuzu networkx pandas

In [2]:
import kuzu
import pandas as pd
import networkx as nx

db = kuzu.Database("../ex_db_kuzu")
conn = kuzu.Connection(db)
kuzu.__version__

'0.7.0'

In [3]:
res = conn.execute(
    """
    MATCH (a1:Account)-[t:Transfer]->(a2:Account)
    RETURN *
    """
)
G = res.get_as_networkx(directed=False)

In [4]:
bc = nx.betweenness_centrality(G, normalized=True)

In [5]:
df = pd.DataFrame.from_dict(bc, orient="index", columns=["betweenness_centrality"])
account_df = df[df.index.str.contains("Account")]
account_df.index = account_df.index.str.replace("Account_", "")
account_df = account_df.reset_index(names=["id"])
account_df.head()

,id,betweenness_centrality
0,1,0.029412
1,5,0.062092
2,3,0.153595
3,2,0.000000
4,9,0.091503


In [6]:
try:
  # Alter original node table schemas to add betweenness_centrality
  conn.execute("ALTER TABLE Account ADD betweenness_centrality DOUBLE DEFAULT 0.0;")
except RuntimeError:
  # If the column already exists, do nothing
  pass

In [7]:
# Copy betweenness_centrality values to Account nodes
x = conn.execute(
  """
  LOAD FROM account_df
  WITH CAST(id AS INT64) AS id, betweenness_centrality
  MERGE (a:Account {id: id})
  ON MATCH SET a.betweenness_centrality = betweenness_centrality
  RETURN
      a.account_id AS account_id,
      a.balance AS balance,
      a.betweenness_centrality AS betweenness_centrality
  ORDER BY betweenness_centrality DESC
  """
)
x.get_as_df().head()


,account_id,balance,betweenness_centrality
0,069392555,5392.0,0.450980
1,082388575,6841.0,0.235294
2,040162526,7115.0,0.202614
3,067853119,5251.0,0.176471
4,080785523,8044.0,0.153595


## Display persons with highest betweenness centrality

In [8]:
res = conn.execute(
    """
    MATCH (p:Person)-[:Owns]-(a:Account)
    RETURN p.name AS person, a.betweenness_centrality AS bc
    ORDER BY bc DESC LIMIT 5;
    """)
res.get_as_df()

,person,bc
0,Amanda,0.450980
1,Edward,0.235294
2,Jon,0.202614
3,Michael,0.176471
4,Carol,0.153595


That's it! We ran the Betweenness Centrality algorithm on the financial network graph and inserted
the values for each account node back into Kùzu.

You can now continue to query the graph and access the
`betweenness_centrality` property on the account nodes to study the results.
